In [53]:
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.client import GoogleCredentials
from datetime import datetime
from dateutil.relativedelta import relativedelta
import os
from dotenv import load_dotenv
import glob

In [54]:
# 1. Defina o caminho e o padrão de busca
CAMINHO_SAIDAS_ETL = r"C:\Users\jea_goncalves\Desktop\bases_balancete\saidas"
padrao_magalu_pkl = "balancete_magalu_processado_*.pkl"
caminho_busca_magalu = os.path.join(CAMINHO_SAIDAS_ETL, padrao_magalu_pkl)

# 2. Encontra todos os arquivos que correspondem ao padrão
arquivos_magalu_encontrados = glob.glob(caminho_busca_magalu)

# 3. Carrega o arquivo mais recente
if arquivos_magalu_encontrados:
    # Encontra o caminho do arquivo com a data de modificação mais recente
    caminho_recente_magalu = max(arquivos_magalu_encontrados, key=os.path.getmtime)
    print(f"Carregando arquivo encontrado: {os.path.basename(caminho_recente_magalu)}")
    
    # Carrega o DataFrame do arquivo pickle
    df_magalu = pd.read_pickle(caminho_recente_magalu)
    
    print("DataFrame 'df_magalu' carregado com sucesso.")
    # print(df_magalu.info()) # Descomente para verificar os tipos de dados
else:
    print(f"ERRO: Nenhum arquivo encontrado com o padrão '{padrao_magalu_pkl}'")
    df_magalu = pd.DataFrame() # Cria um DataFrame vazio para evitar erros futuros


Carregando arquivo encontrado: balancete_magalu_processado_20250630_175117.pkl
DataFrame 'df_magalu' carregado com sucesso.


In [55]:
# Célula 4: Carregando os Balancetes da Época (.pkl)

# (Não precisa reimportar as bibliotecas se já executou a célula anterior)

# --- CARREGAR BALANCETE ÉPOCA (MÊS ANTERIOR) ---
padrao_epoca_anterior_pkl = "balancete_epoca_mes_anterior_*.pkl" # Exemplo de padrão
caminho_busca_epoca_ant = os.path.join(CAMINHO_SAIDAS_ETL, padrao_epoca_anterior_pkl)
arquivos_epoca_ant_encontrados = glob.glob(caminho_busca_epoca_ant)

if arquivos_epoca_ant_encontrados:
    caminho_recente_epoca_ant = max(arquivos_epoca_ant_encontrados, key=os.path.getmtime)
    print(f"Carregando arquivo encontrado: {os.path.basename(caminho_recente_epoca_ant)}")
    df_epoca_anterior = pd.read_pickle(caminho_recente_epoca_ant)
    print("DataFrame 'df_epoca_anterior' carregado com sucesso.\n")
else:
    print(f"AVISO: Nenhum arquivo encontrado com o padrão '{padrao_epoca_anterior_pkl}'")
    df_epoca_anterior = pd.DataFrame()

# --- CARREGAR BALANCETE ÉPOCA (MÊS ATUAL) ---
padrao_epoca_atual_pkl = "balancete_epoca_mes_atual_*.pkl" # Exemplo de padrão
caminho_busca_epoca_atual = os.path.join(CAMINHO_SAIDAS_ETL, padrao_epoca_atual_pkl)
arquivos_epoca_atual_encontrados = glob.glob(caminho_busca_epoca_atual)

if arquivos_epoca_atual_encontrados:
    caminho_recente_epoca_atual = max(arquivos_epoca_atual_encontrados, key=os.path.getmtime)
    print(f"Carregando arquivo encontrado: {os.path.basename(caminho_recente_epoca_atual)}")
    df_epoca_atual = pd.read_pickle(caminho_recente_epoca_atual)
    print("DataFrame 'df_epoca_atual' carregado com sucesso.")
else:
    print(f"AVISO: Nenhum arquivo encontrado com o padrão '{padrao_epoca_atual_pkl}'")
    df_epoca_atual = pd.DataFrame()

Carregando arquivo encontrado: balancete_epoca_mes_anterior_20250714_161800.pkl
DataFrame 'df_epoca_anterior' carregado com sucesso.

Carregando arquivo encontrado: balancete_epoca_mes_atual_20250714_161818.pkl
DataFrame 'df_epoca_atual' carregado com sucesso.


In [56]:
load_dotenv()
CAMINHO_CREDENCIAL_JSON = os.getenv('GOOGLE_CREDENTIALS_SERVICE_ACCOUNT')
ID_PLANILHA = os.getenv('GOOGLE_SHEET_ID')
NOME_ABA  = os.getenv('WORKSHEET_NAME')
TOKEN_USER = os.getenv('AUTHORIZED_USER_FILE_PATH')
gc = gspread.service_account(
    filename=CAMINHO_CREDENCIAL_JSON)

In [58]:
ws = gc.open_by_key(ID_PLANILHA)
aba = ws.worksheet(NOME_ABA)
data = aba.get_all_records()
df = pd.DataFrame(data)
df.head(10000)

,CONCATENAÇÃO,LIVRO,ULTIMA_ALTERCAO_GL,DATA_EFETIVA,PERIODO,EMPRESA,CONTA,DESCRICAO_CONTA,SALDO_INICIAL,DEBITO,CREDITO,SALDO_FINAL,MOV_MES
0,1/1101010001,ML_CORPORATIVO,3/6/2025 08:28:14,01/05/2025,05-25,1,1101010001,CAIXA,"126.187.324,38","800.955.945,69","801.610.449,99","125.532.820,08",-654.5043
1,1/1101010002,ML_CORPORATIVO,26/4/2025 00:15:30,01/05/2025,05-25,1,1101010002,CAIXA ADMINISTRATIVO,0,0,0,0,0
2,1/1101010003,ML_CORPORATIVO,26/4/2025 00:15:30,01/05/2025,05-25,1,1101010003,CAIXA - FECHAMENTO,0,0,0,0,0
3,1/1101010004,ML_CORPORATIVO,26/4/2025 00:15:30,01/05/2025,05-25,1,1101010004,CAIXA - MIGRACAO MAIA,0,0,0,0,0
4,1/1101010005,ML_CORPORATIVO,26/4/2025 00:15:30,01/05/2025,05-25,1,1101010005,CARTÃO PRE-PAGO,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8793,2/3701020002,ML_CORPORATIVO,,01/06/2025,06-25,2,3701020002,IRPJ,"-3.054.456,05",0,0,"-3.054.456,05",0
8794,2/3801010001,ML_CORPORATIVO,,01/06/2025,06-25,2,3801010001,PARTICIPACOES NOS LUCROS - EMPREGADOS,-239.904,0,0,-239.904,0
8795,2/3999999999,ML_CORPORATIVO,,01/06/2025,06-25,2,3999999999,ENCERRAMENTO DO EXERCICIO - RESULTADOS,0,0,0,0,0
8796,2/4105030001,ML_CORPORATIVO,,01/06/2025,06-25,2,4105030001,FOLHA DE PAGTO - CARGOS,0,0,0,0,0


In [59]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

def obter_strings_de_mes():
    """Calcula dinamicamente as strings para o mês atual e anterior no formato 'MM-YY'."""
    hoje = datetime.now()
    mes_atual = hoje.strftime("%m-%y")
    
    mes_anterior_data = hoje - relativedelta(months=1)
    mes_anterior = mes_anterior_data.strftime("%m-%y")
    
    return mes_atual, mes_anterior

def encontrar_arquivo_recente(padrao):
    """Encontra o arquivo mais recente que corresponde a um padrão de nome."""
    try:
        lista_arquivos = glob.glob(padrao)
        if not lista_arquivos:
            print(f"Aviso: Nenhum arquivo encontrado para o padrão: {padrao}")
            return None
        arquivo_mais_recente = max(lista_arquivos, key=os.path.getctime)
        return arquivo_mais_recente
    except Exception as e:
        print(f"Erro ao buscar arquivo com padrão '{padrao}': {e}")
        return None

# Obtendo os meses
mes_atual_str, mes_anterior_str = obter_strings_de_mes()
print(f"Mês Atual: {mes_atual_str}, Mês Anterior: {mes_anterior_str}")

Mês Atual: 07-25, Mês Anterior: 06-25


In [60]:
# Converte colunas para string para garantir a consistência nas comparações futuras
df['EMPRESA'] = df['EMPRESA'].astype(str)
df['PERIODO'] = df['PERIODO'].astype(str)

# --- Carregar arquivos .pkl processados pelo ETL ---

# Defina o diretório onde os arquivos .pkl foram salvos
CAMINHO_SAIDAS_ETL = r"C:\Users\jea_goncalves\Desktop\bases_balancete\saidas"

# 1. Carregar Balancete Magalu
padrao_magalu_pkl = "balancete_magalu_processado_*.pkl"
caminho_busca_magalu = os.path.join(CAMINHO_SAIDAS_ETL, padrao_magalu_pkl)
arquivos_magalu_encontrados = glob.glob(caminho_busca_magalu)

if arquivos_magalu_encontrados:
    caminho_recente_magalu = max(arquivos_magalu_encontrados, key=os.path.getmtime)
    print(f"Carregando arquivo Magalu: {os.path.basename(caminho_recente_magalu)}")
    df_magalu = pd.read_pickle(caminho_recente_magalu)
else:
    print(f"ERRO: Nenhum arquivo encontrado com o padrão '{padrao_magalu_pkl}'")
    df_magalu = pd.DataFrame()

# 2. Carregar Balancete Época (Mês Anterior)
padrao_epoca_anterior_pkl = "balancete_epoca_mes_anterior_*.pkl"
caminho_busca_epoca_ant = os.path.join(CAMINHO_SAIDAS_ETL, padrao_epoca_anterior_pkl)
arquivos_epoca_ant_encontrados = glob.glob(caminho_busca_epoca_ant)

if arquivos_epoca_ant_encontrados:
    caminho_recente_epoca_ant = max(arquivos_epoca_ant_encontrados, key=os.path.getmtime)
    print(f"Carregando arquivo Época (Mês Anterior): {os.path.basename(caminho_recente_epoca_ant)}")
    df_epoca_anterior = pd.read_pickle(caminho_recente_epoca_ant)
else:
    print(f"AVISO: Nenhum arquivo encontrado com o padrão '{padrao_epoca_anterior_pkl}'")
    df_epoca_anterior = pd.DataFrame()

# 3. Carregar Balancete Época (Mês Atual)
padrao_epoca_atual_pkl = "balancete_epoca_mes_atual_*.pkl"
caminho_busca_epoca_atual = os.path.join(CAMINHO_SAIDAS_ETL, padrao_epoca_atual_pkl)
arquivos_epoca_atual_encontrados = glob.glob(caminho_busca_epoca_atual)

if arquivos_epoca_atual_encontrados:
    caminho_recente_epoca_atual = max(arquivos_epoca_atual_encontrados, key=os.path.getmtime)
    print(f"Carregando arquivo Época (Mês Atual): {os.path.basename(caminho_recente_epoca_atual)}")
    df_epoca_atual = pd.read_pickle(caminho_recente_epoca_atual)
else:
    print(f"AVISO: Nenhum arquivo encontrado com o padrão '{padrao_epoca_atual_pkl}'")
    df_epoca_atual = pd.DataFrame()

Carregando arquivo Magalu: balancete_magalu_processado_20250630_175117.pkl
Carregando arquivo Época (Mês Anterior): balancete_epoca_mes_anterior_20250714_161800.pkl
Carregando arquivo Época (Mês Atual): balancete_epoca_mes_atual_20250714_161818.pkl


In [61]:
from datetime import datetime # Certifique-se de que datetime está importado

def atualizar_secao_balancete(df, df_origem, mes_str, tipo_empresa):
    """
    Função principal para atualizar uma seção do balancete na planilha.
    Versão final com o cálculo correto da data_efetiva para a Época.
    """
    # Trava de segurança para não processar meses muito antigos
    if mes_str not in [mes_atual_str, mes_anterior_str]:
        print(f"\n--- AVISO: O período '{mes_str}' está fora da janela de atualização. Pulando o processamento para '{tipo_empresa}'.")
        return df

    print(f"\n--- Iniciando atualização para: {tipo_empresa.upper()} | Mês: {mes_str} ---")
    
    if df_origem.empty:
        print("Aviso: DataFrame de origem está vazio. Pulando esta atualização.")
        return df

    df_fonte = df_origem.copy()
    df_atualizado = df.copy()

    # 1. Padroniza os nomes das colunas da fonte
    if tipo_empresa == 'magalu':
        mapa_colunas = {'periodo': 'periodo', 'empresa': 'empresa', 'conta': 'conta', 'descricao_conta': 'descricao_conta', 'saldo_inicial': 'saldo_inicial', 'debito': 'debito', 'credito': 'credito', 'saldo_final': 'saldo_final'}
    else: # epoca
        mapa_colunas = {'Classificação': 'conta', 'Nome': 'descricao_conta', 'Saldo Anterior': 'saldo_inicial', 'Débito': 'debito', 'Crédito': 'credito', 'Saldo Atual': 'saldo_final'}
    
    df_fonte = df_fonte.rename(columns=mapa_colunas)
    if 'empresa' not in df_fonte.columns: df_fonte['empresa'] = '2'
    if 'periodo' not in df_fonte.columns: df_fonte['periodo'] = mes_str
    df_fonte['empresa'] = df_fonte['empresa'].astype(str)

    # 2. Filtra o bloco de destino e ajusta as linhas (lógica mantida)
    if tipo_empresa == 'magalu':
        filtro_df = (df_atualizado['periodo'] == mes_str) & (df_atualizado['empresa'] != '2')
    else:
        filtro_df = (df_atualizado['periodo'] == mes_str) & (df_atualizado['empresa'] == '2')
    df_bloco_original = df_atualizado[filtro_df]

    ponto_de_insercao = -1
    if not df_bloco_original.empty:
        ponto_de_insercao = df_bloco_original.index[-1]
    else:
        if tipo_empresa == 'epoca':
            filtro_magalu_irmao = (df_atualizado['periodo'] == mes_str) & (df_atualizado['empresa'] != '2')
            bloco_magalu_irmao = df_atualizado[filtro_magalu_irmao]
            if not bloco_magalu_irmao.empty:
                ponto_de_insercao = bloco_magalu_irmao.index[-1]
        if ponto_de_insercao == -1:
            registros_anteriores = df_atualizado[df_atualizado['periodo'] < mes_str]
            if not registros_anteriores.empty:
                ponto_de_insercao = registros_anteriores.index.max()
    
    n_linhas_fonte = len(df_fonte)
    diferenca = n_linhas_fonte - len(df_bloco_original)

    if diferenca > 0:
        novas_linhas_df = pd.DataFrame([{} for _ in range(diferenca)], columns=df_atualizado.columns)
        df_antes = df_atualizado.loc[:ponto_de_insercao]
        df_depois = df_atualizado.loc[ponto_de_insercao+1:]
        df_atualizado = pd.concat([df_antes, novas_linhas_df, df_depois], ignore_index=True)
    elif diferenca < 0:
        indices_para_remover = df_bloco_original.index[n_linhas_fonte:]
        df_atualizado = df_atualizado.drop(indices_para_remover).reset_index(drop=True)

    # 3. Colagem final
    if n_linhas_fonte > 0:
        idx_inicio = df_bloco_original.index[0] if not df_bloco_original.empty else ponto_de_insercao + 1
        indices_destino = range(idx_inicio, idx_inicio + n_linhas_fonte)
        
        colunas_para_colar = [col for col in df_fonte.columns if col in df_atualizado.columns]
        df_fonte.index = indices_destino
        df_atualizado.loc[indices_destino, colunas_para_colar] = df_fonte[colunas_para_colar]
    
        # ==============================================================================
        # 4. PREENCHIMENTO DE DADOS E FÓRMULAS
        # ==============================================================================
        
        # Pre-calcula a data efetiva para o bloco da Época, se aplicável
        data_efetiva_formatada = ""
        if tipo_empresa == 'epoca':
            try:
                # Converte 'MM-YY' para um objeto data e depois para o formato 'DD/MM/YYYY'
                data_obj = datetime.strptime(f"01-{mes_str}", "%d-%m-%y")
                data_efetiva_formatada = data_obj.strftime("%d/%m/%Y")
            except ValueError:
                print(f"AVISO: Não foi possível formatar a data para o período '{mes_str}'.")

        for idx in indices_destino:
            df_atualizado.loc[idx, 'concatenação'] = f"=CONCATENATE(E{idx+2}, G{idx+2}, B{idx+2})"
            
            if tipo_empresa == 'epoca':
                df_atualizado.loc[idx, 'empresa'] = '2'
                df_atualizado.loc[idx, 'periodo'] = mes_str
                
                if 'mov_mes' in df_atualizado.columns:
                    df_atualizado.loc[idx, 'mov_mes'] = f"=J{idx+2}-K{idx+2}"
                
                # Preenche a data efetiva com o valor calculado
                if 'data_efetiva' in df_atualizado.columns and data_efetiva_formatada:
                    df_atualizado.loc[idx, 'data_efetiva'] = data_efetiva_formatada
                
                # Copia o 'livro' da linha de cima, pois não vem na origem
                if idx > 0 and 'livro' in df_atualizado.columns:
                    df_atualizado.loc[idx, 'livro'] = df_atualizado.loc[idx-1, 'livro']

    print("Atualização concluída.")
    return df_atualizado

In [64]:
# --- ORDEM DE EXECUÇÃO DAS ATUALIZAÇÕES ---

# Verifica se o DataFrame principal 'df' foi carregado antes de continuar.
if 'df' in locals() and not df.empty:
    print("Iniciando o processo de atualização da planilha...")

    # Começa com uma cópia do DataFrame original do Google Sheets
    df_atualizado = df.copy()

    # --- AJUSTE: Garante que as colunas dos DataFrames estejam em minúsculas ---
    # Isso evita erros de KeyError se as colunas vierem com letras maiúsculas
    # tanto do Google Sheets (df_atualizado) quanto do Oracle (df_magalu).
    df_atualizado.columns = [col.lower() for col in df_atualizado.columns]
    
    if not df_magalu.empty:
        df_magalu.columns = [col.lower() for col in df_magalu.columns]


    # 1. Magalu - Mês Anterior
    # Filtra o df_magalu para passar apenas os dados do mês anterior
    df_magalu_anterior = df_magalu[df_magalu['periodo'] == mes_anterior_str]
    df_atualizado = atualizar_secao_balancete(df_atualizado, df_magalu_anterior, mes_anterior_str, 'magalu')

    # 2. Época - Mês Anterior
    df_atualizado = atualizar_secao_balancete(df_atualizado, df_epoca_anterior, mes_anterior_str, 'epoca')

    # 3. Magalu - Mês Atual
    # Filtra o df_magalu para passar apenas os dados do mês atual
    df_magalu_atual = df_magalu[df_magalu['periodo'] == mes_atual_str]
    df_atualizado = atualizar_secao_balancete(df_atualizado, df_magalu_atual, mes_atual_str, 'magalu')

    # 4. Época - Mês Atual
    df_atualizado = atualizar_secao_balancete(df_atualizado, df_epoca_atual, mes_atual_str, 'epoca')

    print("\n-------------------------------------------")
    print("Processo de atualização finalizado.")
    print("Pré-visualização do DataFrame final antes de enviar para o Google Sheets:")
    display(df_atualizado.head())
    display(df_atualizado.tail())

else:
    print("ERRO: O DataFrame 'df' não foi carregado corretamente. A execução foi interrompida.")

Iniciando o processo de atualização da planilha...

--- Iniciando atualização para: MAGALU | Mês: 06-25 ---


C:\Users\jea_goncalves\AppData\Local\Temp\ipykernel_14088\217490497.py:73: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['1101010001' '1101010001' '1101010001' ... '4401010005' '4401010009'
 '4401010010']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_atualizado.loc[indices_destino, colunas_para_colar] = df_fonte[colunas_para_colar]


Atualização concluída.

--- Iniciando atualização para: EPOCA | Mês: 06-25 ---
Atualização concluída.

--- Iniciando atualização para: MAGALU | Mês: 07-25 ---
Aviso: DataFrame de origem está vazio. Pulando esta atualização.

--- Iniciando atualização para: EPOCA | Mês: 07-25 ---
Atualização concluída.

-------------------------------------------
Processo de atualização finalizado.
Pré-visualização do DataFrame final antes de enviar para o Google Sheets:


,concatenação,livro,ultima_altercao_gl,data_efetiva,periodo,empresa,conta,descricao_conta,saldo_inicial,debito,credito,saldo_final,mov_mes
0,1/1101010001,ML_CORPORATIVO,3/6/2025 08:28:14,01/05/2025,05-25,1,1101010001.0,CAIXA,"126.187.324,38","800.955.945,69","801.610.449,99","125.532.820,08",-654.5043
1,1/1101010002,ML_CORPORATIVO,26/4/2025 00:15:30,01/05/2025,05-25,1,1101010002.0,CAIXA ADMINISTRATIVO,0,0,0,0,0
2,1/1101010003,ML_CORPORATIVO,26/4/2025 00:15:30,01/05/2025,05-25,1,1101010003.0,CAIXA - FECHAMENTO,0,0,0,0,0
3,1/1101010004,ML_CORPORATIVO,26/4/2025 00:15:30,01/05/2025,05-25,1,1101010004.0,CAIXA - MIGRACAO MAIA,0,0,0,0,0
4,1/1101010005,ML_CORPORATIVO,26/4/2025 00:15:30,01/05/2025,05-25,1,1101010005.0,CARTÃO PRE-PAGO,0,0,0,0,0


,concatenação,livro,ultima_altercao_gl,data_efetiva,periodo,empresa,conta,descricao_conta,saldo_inicial,debito,credito,saldo_final,mov_mes
9349,"=CONCATENATE(E9351, G9351, B9351)",ML_CORPORATIVO,NaN,01/07/2025,07-25,2,3701020002,IRPJ,-4058633.9,0.0,0.0,-4058633.9,=J9351-K9351
9350,"=CONCATENATE(E9352, G9352, B9352)",ML_CORPORATIVO,NaN,01/07/2025,07-25,2,3801010001,PARTICIPACOES NOS LUCROS - EMPREGADOS,-142420.95,0.0,0.0,-142420.95,=J9352-K9352
9351,"=CONCATENATE(E9353, G9353, B9353)",ML_CORPORATIVO,NaN,01/07/2025,07-25,2,3999999999,ENCERRAMENTO DO EXERCICIO - RESULTADOS,0.0,0.0,0.0,0.0,=J9353-K9353
9352,"=CONCATENATE(E9354, G9354, B9354)",ML_CORPORATIVO,NaN,01/07/2025,07-25,2,4105030001,FOLHA DE PAGTO - CARGOS,0.0,0.0,0.0,0.0,=J9354-K9354
9353,"=CONCATENATE(E9355, G9355, B9355)",ML_CORPORATIVO,NaN,01/07/2025,07-25,2,4105030002,FOLHA DE PAGTO - CARGOS,0.0,0.0,0.0,0.0,=J9355-K9355


In [66]:
# ATENÇÃO: Esta célula irá substituir os dados na sua aba do Google Sheets.

print("Iniciando a atualização final da planilha no Google Sheets...")

try:
    # 1. Limpa a aba para garantir uma escrita limpa.
    aba.clear()
    print("Aba da planilha limpa com sucesso.")

    # 2. Cola o DataFrame COMPLETO (df_atualizado), que já contém os meses
    #    anteriores preservados e os meses novos atualizados.
    set_with_dataframe(aba, df_atualizado, include_index=False, resize=True, allow_formulas=True)

    print("\nPlanilha atualizada com sucesso!")
    print("O processo de automação foi concluído.")

except Exception as e:
    print(f"\nOcorreu um erro ao tentar atualizar o Google Sheets: {e}")

Iniciando a atualização final da planilha no Google Sheets...
Aba da planilha limpa com sucesso.

Planilha atualizada com sucesso!
O processo de automação foi concluído.
